## Implementing Retrieval-Augmented Generation (RAG) with Gemini and Pinecone

This approach combines **retrieval** and **generation** techniques to answer questions using external knowledge stored in a vector database.

### Steps:

1. **Data Retrieval from Web**:
   - We use the `BeautifulSoupWebReader` to scrape data from a specified URL, which in this case is related to the Harappan Civilization.
   - The data retrieved is then processed and stored as documents.

2. **Embedding and Vector Store Setup**:
   - The documents are embedded using **Gemini's embedding model** and stored in a **Pinecone vector database**.
   - Pinecone is utilized for efficient indexing and retrieval of documents based on their vector representations.

3. **Query Processing**:
   - After indexing the data in Pinecone, a query engine is set up using **Gemini** (for both embedding and answering queries).
   - The query engine allows the system to retrieve relevant documents from the vector store and generate responses based on them.

### Example Query:
The system answers queries like "By what time Harappan civilization declined?" by retrieving relevant documents and generating a response.

### RAG Architecture:
- **Retrieval**: The relevant data is retrieved from the vector store (Pinecone).
- **Generation**: The Gemini model generates a response based on the retrieved data, providing a complete and context-aware answer.

This setup allows for **accurate, context-driven responses** by leveraging both retrieval from the vector store and generation using the Gemini model.

---

You can modify the code and data sources to suit different domains or questions. The power of RAG lies in its ability to augment the response with external, dynamic knowledge.


In [2]:
!pip install llama-index llama-index-llms-gemini llama-index-embeddings-gemini llama-index-vector-stores-pinecone BeautifulSoup4 google-generativeai pinecone-client


  Using cached grpcio-1.68.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.9 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 579.1 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 502.6 kB/s eta 0:00:000:00:010:00:01:02
Using cached grpcio-1.68.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.9 MB)
  Attempting uninstall: protobuf
    Found existing instal

In [3]:
from llama_index.core import download_loader

In [4]:
BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")

/tmp/ipykernel_800737/3502898812.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")


  Using cached html2text-2024.2.26-py3-none-any.whl
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 518.0 kB/s eta 0:00:00m eta 0:00:010:01:02
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 356.8 kB/s eta 0:00:00m eta 0:00:010:00:04
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=c5c6727d5299886488813299b01f80a83964561bd8ef4ec5614761ad6a197b01
  Stored in directory: /home/rahul/.cache/pip/wheels/c8/d6/6c/384f58df48c00b

In [23]:
loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=["https://en.wikipedia.org/wiki/Harappa"])

In [ ]:
import os
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['PINECONE_API_KEY']=os.getenv("PINECONE_API_KEY")
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings

llm =  Gemini()
gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")
Settings.llm = llm
Settings.embed_model = gemini_embed_model
# gemini_embed_model.

In [ ]:
from pinecone import Pinecone

pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pinecone_index = pinecone_client.Index("pdfq")
print(pinecone_index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [30]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex

vectore_store = PineconeVectorStore(pinecone_index=pinecone_index)
index = VectorStoreIndex.from_documents(documents=documents,vectore_store=vectore_store)
query_engine = index.as_query_engine()
response = query_engine.query("Relation of Mohenjodaro and Harappa")
print(response)

Mohenjo-daro and Harappa are both ruins and are related to the Indus Valley Civilization.



In [31]:
print(pinecone_index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
